In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from tensorflow.keras.models import Sequential, save_model, load_model
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.utils import get_custom_objects
import joblib

# Register custom metric with Keras
def f1_metric(y_true, y_pred):
    y_pred = K.round(y_pred)
    tp = K.sum(K.cast(y_true * y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1 - y_true) * (1 - y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1 - y_true) * y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true * (1 - y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2 * p * r / (p + r + K.epsilon())
    return K.mean(f1)

get_custom_objects().update({"f1_metric": f1_metric})

# Global Variables
dropout_value = 0.5
epochs_value = 10000
patience_value = 100
random_state_value = 50

# Load the training data
print("Loading training data...")
train_file_path = 'train.csv'  # replace with your actual file path
train_data = pd.read_csv(train_file_path)
print("Training data loaded.")

# Separate features and target variable
X_train = train_data.drop(columns=['Target'])
y_train = train_data['Target']

# Handle missing values
X_train = X_train.fillna(X_train.mean())

# Normalize the features
print("Normalizing features...")
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
print("Features normalized.")

# Build the neural network model
print("Building neural network model...")
def build_nn_model(input_dim):
    model = Sequential()
    model.add(Dense(128, input_dim=input_dim, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(dropout_value))
    model.add(Dense(64, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(dropout_value))
    model.add(Dense(32, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(dropout_value))
    model.add(Dense(16, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=[f1_metric])
    return model

# Initialize the neural network model
nn_model = build_nn_model(X_train_scaled.shape[1])

# Train the neural network model
print("Training neural network model...")
early_stopping = EarlyStopping(monitor='val_loss', patience=patience_value, restore_best_weights=True, min_delta=0.001)
model_checkpoint = ModelCheckpoint('best_nn_model.h5', monitor='val_loss', save_best_only=True, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001, verbose=1)

history = nn_model.fit(X_train_scaled, y_train, validation_split=0.2, epochs=epochs_value, batch_size=32, callbacks=[early_stopping, model_checkpoint, reduce_lr])

# Load the best neural network model
print("Loading best neural network model...")
nn_model.load_weights('best_nn_model.h5')

# Save the neural network model
print("Saving neural network model...")
nn_model_path = "neural_network_model.h5"
save_model(nn_model, nn_model_path)

# Initialize the BaggingClassifier with best parameters
print("Initializing BaggingClassifier with best parameters...")
bagging_clf = BaggingClassifier(
    base_estimator=RandomForestClassifier(
        criterion='gini',
        max_depth=8,
        max_features='log2',
        n_estimators=100,
        random_state=random_state_value
    ),
    n_estimators=10,
    random_state=random_state_value
)

# Train the BaggingClassifier
print("Training BaggingClassifier...")
bagging_clf.fit(X_train_scaled, y_train)

# Save the BaggingClassifier model
print("Saving BaggingClassifier model...")
bagging_model_path = "bagging_classifier_model.pkl"
joblib.dump(bagging_clf, bagging_model_path)

# Load the test data
print("Loading test data...")
test_file_path = 'test.csv'  # replace with your actual file path
test_data = pd.read_csv(test_file_path)
print("Test data loaded.")

# Handle missing values in the test data
X_test = test_data.fillna(test_data.mean())

# Normalize the test data
print("Normalizing test data...")
X_test_scaled = scaler.transform(X_test)
print("Test data normalized.")

# Make predictions with the BaggingClassifier model
print("Making predictions with the BaggingClassifier model...")
y_pred_bagging = bagging_clf.predict(X_test_scaled)

# Save predictions to a CSV file
print("Saving predictions...")
output = pd.DataFrame({'ID': test_data['ID'], 'Target': y_pred_bagging})
output.to_csv('predictions.csv', index=False)
print("Predictions saved to predictions.csv.")


Loading training data...
Training data loaded.
Normalizing features...
Features normalized.
Building neural network model...
Training neural network model...
Epoch 1/10000
53/67 [======================>.......] - ETA: 0s - loss: 0.8940 - f1_metric: 0.5681 
Epoch 1: val_loss improved from inf to 0.68983, saving model to best_nn_model.h5
67/67 [==============================] - 2s 6ms/step - loss: 0.8866 - f1_metric: 0.5696 - val_loss: 0.6898 - val_f1_metric: 0.6830 - lr: 0.0010
Epoch 2/10000
52/67 [======================>.......] - ETA: 0s - loss: 0.8078 - f1_metric: 0.5741

C:\Users\bapti\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 2: val_loss did not improve from 0.68983
67/67 [==============================] - 0s 3ms/step - loss: 0.7997 - f1_metric: 0.5807 - val_loss: 0.6970 - val_f1_metric: 0.6814 - lr: 0.0010
Epoch 3/10000
55/67 [=======================>......] - ETA: 0s - loss: 0.7380 - f1_metric: 0.6237
Epoch 3: val_loss did not improve from 0.68983
67/67 [==============================] - 0s 2ms/step - loss: 0.7377 - f1_metric: 0.6289 - val_loss: 0.7000 - val_f1_metric: 0.6719 - lr: 0.0010
Epoch 4/10000
52/67 [======================>.......] - ETA: 0s - loss: 0.7369 - f1_metric: 0.6104
Epoch 4: val_loss did not improve from 0.68983
67/67 [==============================] - 0s 3ms/step - loss: 0.7407 - f1_metric: 0.6017 - val_loss: 0.6996 - val_f1_metric: 0.6592 - lr: 0.0010
Epoch 5/10000
51/67 [=====================>........] - ETA: 0s - loss: 0.7135 - f1_metric: 0.6369
Epoch 5: val_loss did not improve from 0.68983
67/67 [==============================] - 0s 3ms/step - loss: 0.7153 - f1_metric: 0.63

Epoch 30/10000
65/67 [============================>.] - ETA: 0s - loss: 0.6962 - f1_metric: 0.6642
Epoch 30: val_loss did not improve from 0.68983
67/67 [==============================] - 0s 3ms/step - loss: 0.6967 - f1_metric: 0.6647 - val_loss: 0.6925 - val_f1_metric: 0.6936 - lr: 1.0000e-04
Epoch 31/10000
46/67 [===================>..........] - ETA: 0s - loss: 0.6833 - f1_metric: 0.6651
Epoch 31: val_loss did not improve from 0.68983
67/67 [==============================] - 0s 3ms/step - loss: 0.6860 - f1_metric: 0.6689 - val_loss: 0.6929 - val_f1_metric: 0.6910 - lr: 1.0000e-04
Epoch 32/10000
61/67 [==========================>...] - ETA: 0s - loss: 0.6937 - f1_metric: 0.6684
Epoch 32: val_loss did not improve from 0.68983
67/67 [==============================] - 0s 3ms/step - loss: 0.6939 - f1_metric: 0.6687 - val_loss: 0.6925 - val_f1_metric: 0.6879 - lr: 1.0000e-04
Epoch 33/10000
46/67 [===================>..........] - ETA: 0s - loss: 0.6990 - f1_metric: 0.6580
Epoch 33: val_lo

Epoch 57/10000
56/67 [========================>.....] - ETA: 0s - loss: 0.6872 - f1_metric: 0.6730
Epoch 57: val_loss did not improve from 0.68783
67/67 [==============================] - 0s 2ms/step - loss: 0.6884 - f1_metric: 0.6755 - val_loss: 0.6882 - val_f1_metric: 0.6836 - lr: 1.0000e-04
Epoch 58/10000
55/67 [=======================>......] - ETA: 0s - loss: 0.6931 - f1_metric: 0.6591
Epoch 58: val_loss improved from 0.68783 to 0.68761, saving model to best_nn_model.h5
67/67 [==============================] - 0s 3ms/step - loss: 0.6925 - f1_metric: 0.6577 - val_loss: 0.6876 - val_f1_metric: 0.6876 - lr: 1.0000e-04
Epoch 59/10000
50/67 [=====================>........] - ETA: 0s - loss: 0.6972 - f1_metric: 0.6545
Epoch 59: val_loss did not improve from 0.68761
67/67 [==============================] - 0s 3ms/step - loss: 0.6970 - f1_metric: 0.6559 - val_loss: 0.6876 - val_f1_metric: 0.6894 - lr: 1.0000e-04
Epoch 60/10000
49/67 [====================>.........] - ETA: 0s - loss: 0.692

...

60/67 [=========================>....] - ETA: 0s - loss: 0.6194 - f1_metric: 0.7161
Epoch 267: val_loss did not improve from 0.65744
67/67 [==============================] - 0s 2ms/step - loss: 0.6173 - f1_metric: 0.7162 - val_loss: 0.6587 - val_f1_metric: 0.6931 - lr: 1.0000e-04
Epoch 268/10000
60/67 [=========================>....] - ETA: 0s - loss: 0.6210 - f1_metric: 0.7054
Epoch 268: val_loss did not improve from 0.65744
67/67 [==============================] - 0s 2ms/step - loss: 0.6185 - f1_metric: 0.7068 - val_loss: 0.6590 - val_f1_metric: 0.6924 - lr: 1.0000e-04
Epoch 269/10000
58/67 [========================>.....] - ETA: 0s - loss: 0.6201 - f1_metric: 0.7116
Epoch 269: val_loss did not improve from 0.65744
67/67 [==============================] - 0s 2ms/step - loss: 0.6198 - f1_metric: 0.7138 - val_loss: 0.6592 - val_f1_metric: 0.6934 - lr: 1.0000e-04
Epoch 270/10000
60/67 [=========================>....] - ETA: 0s - loss: 0.6116 - f1_metric: 0.7238
Epoch 270: val_loss did n

Epoch 295/10000
61/67 [==========================>...] - ETA: 0s - loss: 0.6111 - f1_metric: 0.7289
Epoch 295: val_loss did not improve from 0.65744
67/67 [==============================] - 0s 2ms/step - loss: 0.6047 - f1_metric: 0.7356 - val_loss: 0.6593 - val_f1_metric: 0.6946 - lr: 1.0000e-04
Epoch 296/10000
61/67 [==========================>...] - ETA: 0s - loss: 0.6027 - f1_metric: 0.7252
Epoch 296: val_loss did not improve from 0.65744
67/67 [==============================] - 0s 2ms/step - loss: 0.5991 - f1_metric: 0.7264 - val_loss: 0.6595 - val_f1_metric: 0.6946 - lr: 1.0000e-04
Epoch 297/10000
60/67 [=========================>....] - ETA: 0s - loss: 0.6047 - f1_metric: 0.7230
Epoch 297: val_loss did not improve from 0.65744
67/67 [==============================] - 0s 2ms/step - loss: 0.6029 - f1_metric: 0.7257 - val_loss: 0.6586 - val_f1_metric: 0.6965 - lr: 1.0000e-04
Epoch 298/10000
57/67 [========================>.....] - ETA: 0s - loss: 0.5981 - f1_metric: 0.7419
Epoch 298

Epoch 323/10000
60/67 [=========================>....] - ETA: 0s - loss: 0.6076 - f1_metric: 0.7108
Epoch 323: val_loss did not improve from 0.65744
67/67 [==============================] - 0s 2ms/step - loss: 0.6087 - f1_metric: 0.7122 - val_loss: 0.6580 - val_f1_metric: 0.7002 - lr: 1.0000e-04
Epoch 324/10000
57/67 [========================>.....] - ETA: 0s - loss: 0.5841 - f1_metric: 0.7489
Epoch 324: val_loss did not improve from 0.65744
67/67 [==============================] - 0s 2ms/step - loss: 0.5837 - f1_metric: 0.7467 - val_loss: 0.6588 - val_f1_metric: 0.6969 - lr: 1.0000e-04
Epoch 325/10000
60/67 [=========================>....] - ETA: 0s - loss: 0.5872 - f1_metric: 0.7412
Epoch 325: val_loss did not improve from 0.65744
67/67 [==============================] - 0s 2ms/step - loss: 0.5901 - f1_metric: 0.7374 - val_loss: 0.6597 - val_f1_metric: 0.6987 - lr: 1.0000e-04
Epoch 326/10000
60/67 [=========================>....] - ETA: 0s - loss: 0.6005 - f1_metric: 0.7244
Epoch 326

C:\Users\bapti\AppData\Local\Temp\ipykernel_1520\1686237341.py:93: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  save_model(nn_model, nn_model_path)
C:\Users\bapti\anaconda3\Lib\site-packages\sklearn\ensemble\_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


Saving BaggingClassifier model...
Loading test data...
Test data loaded.
Normalizing test data...
Test data normalized.
Making predictions with the BaggingClassifier model...
Saving predictions...
Predictions saved to predictions.csv.
